In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv("../.env")

client = OpenAI(api_key=os.environ['API_KEY'])

MODEL_NAME = "gpt-4o-mini"


In [10]:
from openai.types.chat import ChatCompletionToolParam

tools = [
    ChatCompletionToolParam(
        {
            "type": "function",
            "function": {
                "name": "set_tags",
                "description": "社内ツールのNPSアンケートから得られたユーザーの「コメントのリスト」から、コメントに対して適切なタグを付与する。",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "comment_tags": {
                            "type": "array",
                            "items": {
                                "comment": {
                                    "type": "string",
                                    "description": "タグつけ対象のコメント"
                                },
                                "positive": {
                                    "type": "string",
                                    "description": "ポジティブなコメント"
                                },
                                "negative": {
                                    "type": "string",
                                    "description": "ネガティブなコメント"
                                },
                            },
                        },
                    },
                    "required": ["comment", "positive", "negative"],
                },
            },
        }
    )
]

In [ ]:
print("処理スタート")

df = pd.read_excel('サンプルデータ.xlsx', sheet_name='コメント')

product_data_text = df.astype(str)
prompt_text = f"""
あなたは、
下記に示す社内ツールのNPSアンケートから得られたユーザーの「コメントのリスト」から、
コメントに対して適切なタグを付与するアシスタントです。

タグは以下の6種類です。
#タグ
positive: ポジティブなコメント
negative: ネガティブなコメント

あなたは以下のルールに従って、コメントに対して適切なタグのみを返します。
あなたは例や説明など補足の情報を出力してはいけません。
# ルール
1. コメントに対して複数のタグを付与できる。
2. 出力は必ず JSON 形式で返してください。JSON 以外の文章は出力しないでください。

# コメントのリスト
{product_data_text}
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt_text},
    ],
    tools=tools,
    tool_choice={
        "type": "function",
        "function": {"name": "set_tags"}
    },
    response_format={"type": "json_object"}
)

tool = response.choices[0].message.tool_calls[0]

data = json.loads(tool.function.arguments)

df_out = pd.DataFrame(data['comment_tags'])

df_out.to_excel("コメント分類.xlsx", index=False)


print("処理終了")



処理スタート
処理終了
